## Fully-programmable universal quantum simulator with a one-dimensional quantum processor

VM Bastidas, T Haug, C Gravel, LC Kwek, WJ Munro, K Nemoto. arXiv:2009.00823

@author: Tobias Haug, comments to tobias.haug@u.nus.edu

This program finds driving protocols to create
arbitrary effective Hamiltonians via periodic driving. The initial Hamiltonian is a one-dimensional nearest-neighbor Bose-Hubbard model in the limit of large interactions. By periodically driving the local potential in time, one can create arbitrary effective Hamiltonians with arbitrary couplings.
A desired Floquet Hamiltonian is found by optimizing the driving protocol.
You can specify a desired effective Hamiltonian
e.g. all-to-all coupling, star-graph, ring with the variable "target_model".


The effective Hamiltonian corresponds to a unitary that is optimised with GRAPE using the qutip library.
This unitary is the Floquet operator, describing the effective dynamics with the effective Hamiltonian.

In [11]:
import numpy as np
import qutip as qt
import scipy as sc
import scipy.sparse as sp
import qutip.control.pulseoptim

import pickle

In [17]:
#class that contains model for nearest-neighbor chain
class particleBasis:
    """
    Many-body basis of specific <nb> charge state on a lattice with <N> sites.
    """
    def __init__(self, N, nb):
        """
        Parameters
        ----------
        N : int
            Number of sites
        nb : int
            Number of bosons
        """
        self.N = N  # number of sites

        self.nb = nb
        if(self.nb>N or self.nb<0):
            raise NameError("Number of particles invalid")



        
        self.len=particleBasis.size(N, nb) #hilbertspace
        self.hilbertspace=self.len

        self.ps = np.sqrt(particleBasis.lowest_primes(N)) #used for hasing the states

        self.vs = self.generate(N,nb) #list of all possible states
        self.hashes = self.hash(self.vs)+10**(-10)  #incorrect rounding of hash function at 15th decimal observed, create offset for searchsorted 
        self.sorter = particleBasis.argsort(self.hashes)
        
        
        #cache correlators
        #only upper triangle is stored, rest by taking adjoint
        self.correlator_list=[[[] for i in range(self.N)] for j in range(self.N)]
        self.correlator_fermion_list=[[[] for i in range(self.N)] for j in range(self.N)]


    #calculates a_n a_m^\dagger operator, select fermion whether has fermionic statistics
    def op_a_n_a_m_dagger(self,n,m,fermion=0):
        if(n!=m):
            Opi,Opj,Opvs=[],[],[]


            nbspace=self.len

            for i in range(nbspace):

                targetstate=np.array(self.vs[i])
                #print(targetstate,n,m)
                if(targetstate[n]==1 and targetstate[m]==0):
                    targetstate[n]=0
                    targetstate[m]=1
                

                    coefficient=1
                    if(fermion==1):
                        coefficient=-2*np.mod(np.sum(self.vs[i][min(n,m)+1:max(n,m)]),2)+1 #Jordan-Wigner string

                    targetindex=self.stateindex(targetstate)
                    Opi.append(i)
                    Opj.append(targetindex)

                    Opvs.append(coefficient)
            
            correlatorOp=sp.coo_matrix((Opvs, (Opi, Opj)), shape=(self.hilbertspace, self.hilbertspace),dtype=complex).tocsr()

            
        else:
            diag=np.zeros(self.hilbertspace,dtype=complex)
            for i in range(self.hilbertspace):
                diag[i] = self.vs[i,n]
            correlatorOp=sp.diags(diag,0,format="csr")
        return correlatorOp
    

    def get_a_n_a_m_dagger(self,n,m,fermion=0):
        """
        check whether correlator has been cached, then return
        """
        #check whether m smaller n, if so then take adjoint
        if(m<n):
            do_adjoint=True
            index1=m
            index2=n
        else:
            do_adjoint=False
            index1=n
            index2=m
            
        if(fermion==0):
            if(self.correlator_list[index1][index2]==[]):
                self.correlator_list[index1][index2]=self.op_a_n_a_m_dagger(index1,index2,fermion=0)
            if(do_adjoint==False):
                return self.correlator_list[index1][index2]
            else:
                return self.correlator_list[index1][index2].transpose().conjugate()
        else:
            if(self.correlator_fermion_list[index1][index2]==[]):
                self.correlator_fermion_list[index1][index2]=self.op_a_n_a_m_dagger(index1,index2,fermion=1)
                
            if(do_adjoint==False):
                return self.correlator_fermion_list[index1][index2]
            else:
                return self.correlator_fermion_list[index1][index2].transpose().conjugate()
    
           
    def stateindex(self,state):
        """
        Converts state to hash and searches for the hash among hashes,
        which are sorted by the sorter list.

        Parameters
        ----------
        state : ndarray
            An array of one or more states
        """  
        key = self.hash(state)
        #inorrect rounding of hashes at the 15th decimal observed
        sortednumber=np.searchsorted(self.hashes, key, sorter=self.sorter)
        targetindex=self.sorter[sortednumber] #finds key in list such that key is before next higher element. Problem may arise if key and closest element are too close, and get interchanged?

        if(np.array_equal(self.vs[targetindex],state)==False):
            print("Found wrong state! To find:",state, " with hash ",key.astype(str)," Found:",self.vs[targetindex], " has hash", (self.hash(self.vs[targetindex])).astype(str))
            
            if(np.array_equal(self.vs[self.sorter[sortednumber-1]],state)==True):
                print("Was actually previous index in sorted array")
                targetindex=self.sorter[sortednumber-1]
            elif(np.array_equal(self.vs[self.sorter[sortednumber+1]],state)==True):
                print("Was actually next index in sorted array")
                targetindex=self.sorter[sortednumber+1]
            else:
                prevIndex=self.sorter[sortednumber-1]
                nextIndex=self.sorter[sortednumber+1]
                print("Index:",targetindex, "Previous", self.vs[prevIndex], self.hash(self.vs[prevIndex]).astype(str), " Next:", self.vs[nextIndex],self.hash(self.vs[nextIndex]).astype(str))
                print("differences of hash", (key-self.hash(self.vs[targetindex])).astype(str))
                raise NameError("Found wrong state!")
        

        return targetindex
    

    def generate(self,N,nb):
        """
        Generate basis incrementally based on the method described in e.g.
        http://iopscience.iop.org/article/10.1088/0143-0807/31/3/016

        Parameters
        ----------
        N : int
            Number of sites
        nb : int
            Number of bosons
        """

        hilbertspace=self.len
        states = np.zeros((hilbertspace, N), dtype=int)


        self.counterstates=0
        #print(N,nb,hilbertspace)
        state=states[0]
        self.get_recursive_states(nb,N,0,state,states)
        if(self.counterstates!=np.shape(states)[0]):
            raise NameError("Not all states accounted for")
        states=np.fliplr(states)
        
        return states

    def get_recursive_states(self,particlesleft,N,currentPosition,state2,list_states):
        ##recursivly find all states with fixed number of particles
        state=np.array(state2)
        if(particlesleft==0):
            #print(state)
            list_states[self.counterstates,:]=state
            self.counterstates=self.counterstates+1
            #print(list_states)

        else:
            if(particlesleft<=N-currentPosition-1):
                self.get_recursive_states(particlesleft,N,currentPosition+1,np.array(state),list_states)
                #print(state,particlesleft,"A")
         
            if(particlesleft>0 and particlesleft<=N-currentPosition):
                p1state=state
                p1state[currentPosition]=1
                #print(p1state,particlesleft)       
                self.get_recursive_states(particlesleft-1,N,currentPosition+1,p1state,list_states)
     
    @staticmethod
    def lowest_primes(n):
        """
        Return the lowest n primes
    
        Parameters
        ----------
        n : int
            Number of primes to return
        """
        return particleBasis.primes(n**2)[:n]

    @staticmethod
    def primes(upto):
        """
        Prime sieve below an <upto> value.
        Copied from http://rebrained.com/?p=458
    
        Parameters
        ----------
        upto : int
            Find all primes leq this limit.
        """
        primes = np.arange(3, upto+1, 2)
    
        isprime = np.ones(int((upto-1)/2), dtype=bool)
    
        for factor in primes[:int(np.sqrt(upto))]:
    
            if isprime[int((factor-2)/2)]:
                isprime[int((int(factor)*3-2)/2)::int(factor)] = 0
    
        return np.insert(primes[isprime], 0, 2)
    



    @staticmethod
    def size(N, nb):
        #get size of hilbertspace
        return int(sc.special.binom(N, nb))


    def hash(self,states):
        """
        Hash function as given in:
        http://iopscience.iop.org/article/10.1088/0143-0807/31/3/016

        Parameters
        ----------
        states : ndarray
            List of states to hash
        """
        #n = self.N

        return states.dot(self.ps)
    
    @staticmethod
    def argsort(hashes):
        """
        Argsort our hashes for searching, using e.g. quicksort.
        """
        return np.argsort(hashes, 0, 'quicksort')

In [18]:

#calculate trace overlap of two unitaries, e.g. fidelity
def traceoverlap(U1,U2):
    d=U1.dims[0][0]
    return 1/d*np.abs((U1.dag()*U2).tr())

In [19]:
##set here properties of quantum chain that is driven as well as desired effective Hamiltonian

systemLength=5 #size of driving chain
particle_number=1 #number of particles in bare system

J=1 #nearest neighbor coupling of bare Hamiltonian
maxampl=5 #maximal amplitude of driving potential of bare Hamiltonian

target_model=0 #topology of couplings of effective Hamiltonian, 0: star configuration 1: all-to-all coupling, 2:ring


target_time=0.35 #time of effective Hamiltonian

ntimesteps=10 #number of timesteps for discrete driving protocol
tmax=10 #time of driving potential



repeatOptimize=20 #repeat optimization procedure to avoid local minima



target_fermion=1 #particle type of desires effective Hamiltonian 0: bosonic, 1: fermionic
max_wall_time=2000 #GRAPE maximal walltime in seconds
total_episodes=10000 #maximal number of episodes per optimisation

In [20]:
#define chain model
physmodel=particleBasis(systemLength,particle_number)

#define bare Hamiltonian of chain with nearest-neighbor coupling
bare_hamiltonian=qt.Qobj(sum([J*(physmodel.get_a_n_a_m_dagger(i,i+1)+physmodel.get_a_n_a_m_dagger(i+1,i)) for i in range(systemLength-1)]))

#define driving (local potential)
driving_hamiltonians=[qt.Qobj(physmodel.get_a_n_a_m_dagger(i,i)) for i in range(systemLength)]



hilbertspace=physmodel.hilbertspace
print("Hilbertspace",hilbertspace)

#connectionMatrix contains all couplings, e.g. connectionMatrix[3,4]=1 means site 3 is coupled to site 4 with strength 1
#it should be a hermition matrix
#targetHamiltonian is constructed from this matrix
connectionMatrix=None
connectionMatrix=np.zeros([systemLength,systemLength])
if(target_model==0): #Star
    center=int(systemLength/2)
    for i in range(1,int(systemLength/2)+1):
        connectionMatrix[center,center+i]=1
        connectionMatrix[center+i,center]=np.conjugate(connectionMatrix[center,center+i])
        connectionMatrix[center,center-i]=1
        connectionMatrix[center-i,center]=np.conjugate(connectionMatrix[center,center-i])

elif(target_model==1): #All to all

    for i in range(systemLength):
        for j in range(i+1,systemLength):
            connectionMatrix[i,j]=1#np.exp(1j*2*np.pi*np.random.rand())
            connectionMatrix[j,i]=np.conj(connectionMatrix[i,j])

elif(target_model==2): #ring coupling
    for i in range(systemLength):
        targetsite=int(np.mod(i+1,systemLength))
        connectionMatrix[i,targetsite]=1
        connectionMatrix[targetsite,i]=np.conj(connectionMatrix[i,targetsite])

#effective Hamiltonian with couplings we want to achieve
targetH=0
for i in range(systemLength):
    for j in range(i+1,systemLength):
        if(connectionMatrix[i,j]!=0):
            targetH+=connectionMatrix[i,j]*physmodel.get_a_n_a_m_dagger(i,j,fermion=target_fermion)+np.conjugate(connectionMatrix[i,j])*physmodel.get_a_n_a_m_dagger(j,i,fermion=target_fermion)

##effective Hamiltonian to be created via driving
targetH=qt.Qobj(targetH)


#unitary corresponding to effective Hamiltonian
targetU=(-1j*targetH*target_time).expm()   


#now find protocol that generates effective Hamiltonian via GRAPE

#define constant (drift) Hamiltonian, and control (time-dependent) Hamiltonians for different settings
ctrls=driving_hamiltonians
drift=bare_hamiltonian


#initial unitary is identity
initial=qt.qeye(hilbertspace)

target_unitary=qt.Qobj(targetU)

#overlap of target unitary with identity
#identOverlap=traceoverlap(initial,target_unitary)


maxfidel=0 #keep track which was best unitary found

counter_iterations=0
counter_funccall=0
counter_walltime=0

#use GRAPE via Qutip library to find protocol. Uses optimization routine to optimize protocol via gradient descent wiht LBFGS.
#repeat optimisation repeatOptimize times and take best result as one may get local minimas
for r in range(repeatOptimize):
    #GRAPE routine of Qutip library
    res=qt.control.pulseoptim.optimize_pulse_unitary(
            drift, ctrls, initial, target_unitary,
            num_tslots=ntimesteps, evo_time=tmax,
            amp_lbound=-maxampl, amp_ubound=maxampl,
            fid_err_targ=1e-10, min_grad=1e-10,
            max_iter=total_episodes, max_wall_time=max_wall_time,
            fid_params={'phase_option':'SU'},
            alg='GRAPE',   
            gen_stats=True
            )


    #optimized unitary
    Uopt=res.evo_full_final
    #fidelity
    fidel=traceoverlap(Uopt,target_unitary) #traceoverlap can be one even for a matrix which is not actually the same....

    print("try:",r,", Fidelity:",fidel,", termination:",res.termination_reason,", number iterations:",res.num_iter)

    counter_iterations+=res.num_iter

    counter_funccall+=res.num_fid_func_calls
    counter_walltime+=res.wall_time



    #check whether at this optimisation run a better Hamiltonian was found via GRAPE than before
    if(fidel>maxfidel):
        maxfidel=fidel
        driving_amplitudes=res.final_amps
        driving_unitary=qt.Qobj(Uopt)


#calculate unitary
U_prop=qt.qeye(hilbertspace)
for j in range(ntimesteps):
    H_prop=drift+sum([ctrls[i]*driving_amplitudes[j,i] for i in range(len(ctrls))])
    U_prop=(-1j*H_prop*tmax/ntimesteps).expm()*U_prop

    

driving_unitary=U_prop

#calculate fidelity between driving and desired unitary
final_fidelity=traceoverlap(driving_unitary,target_unitary)


Hilbertspace 5
try: 0 , Fidelity: 0.9999999982577485 , termination: function converged , number iterations: 33
try: 1 , Fidelity: 0.999999998755359 , termination: function converged , number iterations: 36
try: 2 , Fidelity: 0.9999999992496691 , termination: function converged , number iterations: 37
try: 3 , Fidelity: 0.9999999973000367 , termination: function converged , number iterations: 32
try: 4 , Fidelity: 0.9999999987972938 , termination: function converged , number iterations: 33
try: 5 , Fidelity: 0.999999995133824 , termination: function converged , number iterations: 61
try: 6 , Fidelity: 0.9999999983756471 , termination: function converged , number iterations: 31
try: 7 , Fidelity: 0.9999999990405528 , termination: function converged , number iterations: 37
try: 8 , Fidelity: 0.9999999981309066 , termination: function converged , number iterations: 36
try: 9 , Fidelity: 0.9999999984214875 , termination: function converged , number iterations: 40
try: 10 , Fidelity: 0.99999

In [21]:
print("Fidelity of found unitary",final_fidelity)

Fidelity of found unitary 0.99999999993675
